In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# WebDriver 실행 (기본 경로 사용)
driver = webdriver.Chrome()

# 웹페이지에 접속
driver.get('https://www.chevrolet.co.kr/chevy/as-reserve.gm') 

# 예약 희망 날짜 선택2 요소가 나타날 때까지 기다림
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, 'reservationDate2'))
)

# 예약 희망 날짜 선택2를 가장 마지막 날짜로 선택
reservation_date_select = driver.find_element(By.ID, 'reservationDate2')
options = reservation_date_select.find_elements(By.TAG_NAME, 'option')
options[-1].click()  # 가장 마지막 옵션 선택

# "검색" 버튼이 나타날 때까지 기다린 후 클릭
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, 'btnSearch'))
).click()

# complete_area가 나타날 때까지 기다림
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, 'complete_area'))
)

# complete_area 내부의 ul li 요소를 찾음
complete_area = driver.find_element(By.CLASS_NAME, 'complete_area')
list_items = complete_area.find_elements(By.CSS_SELECTOR, "ul li")

# 업체 정보 수집
store_names = []
addresses = []
phone_numbers = []

for item in list_items:
    # 업체명
    name = item.find_element(By.CSS_SELECTOR, "p.center span").text
    # 주소
    address = item.find_element(By.CSS_SELECTOR, "p.adress span").text
    # 전화번호
    phone = item.find_element(By.CSS_SELECTOR, "p.tel span").text
    
    store_names.append(name)
    addresses.append(address)
    phone_numbers.append(phone)

# DataFrame 생성
df = pd.DataFrame({
    '업체명': store_names,
    '주소': addresses,
    '전화번호': phone_numbers
})

# DataFrame 확인
print(df)

           업체명                             주소            전화번호
0      강서바로서비스           서울 강서구 까치산로 63 (화곡동)    02-2663-0400
1     동일산서비스센터      경기 고양시 일산동구 성현로 195 (성석동)    031-977-8779
2      덕양바로서비스  경기 고양시 덕양구 은빛로17번길 5-29 (화정동)    031-968-8222
3      역삼지정서비스        서울 강남구 언주로 108길 6 (역삼동)     02-538-0085
4      신진지정서비스        서울 금천구 남부순환로 1408 (독산동)     02-855-8204
..         ...                            ...             ...
341    천안서비스센터     충남 천안시 서북구 두정공단1길 51 (두정동)  041-555-6182~3
342    홍성서비스센터     충남 홍성군 홍성읍 충서로 1216번길 6-34  041-634-8254~5
343    논산바로서비스            충남 논산시 시민로 106 (내동)    041-735-8255
344   남천안서비스센터       충남 천안시 동남구 충절로 455 (삼룡동)    041-567-7288
345  포항상도지정서비스               경북 포항시 남구 중섬로 65    054-282-1614

[346 rows x 3 columns]


In [19]:
df['주소'] = df['주소'].str.replace('특별시|광역시|특별자치도|특별자치시', '', regex=True)
df['주소'] = df['주소'].str.replace('경기도', '경기', regex=False)
df['주소'] = df['주소'].str.replace('강원도', '강원', regex=False)
df['주소'] = df['주소'].str.replace('충청북도', '충북', regex=False)
df['주소'] = df['주소'].str.replace('충청남도', '충남', regex=False)
df['주소'] = df['주소'].str.replace('전라북도', '전북', regex=False)
df['주소'] = df['주소'].str.replace('전라남도', '전남', regex=False)
df['주소'] = df['주소'].str.replace('경상북도', '경북', regex=False)
df['주소'] = df['주소'].str.replace('경상남도', '경남', regex=False)

# '시도' 열에 주소 첫 두 글자 추가
df['시도'] = df['주소'].str[:2]

# '시군구' 열에 첫 번째와 두 번째 띄어쓰기 사이의 텍스트 추출
df['시군구'] = df['주소'].str.extract(r'^\S+\s(\S+)')[0]

df['시도+시군구'] = df['시도'] + df['시군구']

In [25]:
df.to_excel('쉐보레_최종.xlsx')